# Create appliances individual consumption dataset

## Download files from figshare

In [2]:
import os
import requests
import re
import zipfile


In [3]:
def getFilename(cd):
    """
    Get filename from content-disposition
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0].strip('"')


def downloadFile(url):
    """
    Download file from URL
    Returns file name
    """
    headers = {"Range": "bytes=0-100"}
    r = requests.get(url, allow_redirects=True, headers=headers)
    filename = getFilename(r.headers.get('content-disposition'))
    if not os.path.exists(filename):
        r = requests.get(url, allow_redirects=True)
        open(filename, 'wb').write(r.content)
    return filename


In [3]:
urls = [
    "https://figshare.com/ndownloader/files/26179625",  # house_00
    "https://figshare.com/ndownloader/files/26179622",  # house_01
    "https://figshare.com/ndownloader/files/26179628",  # house_02
    "https://figshare.com/ndownloader/files/26179640",  # house_03
    "https://figshare.com/ndownloader/files/26179643",  # house_04
    "https://figshare.com/ndownloader/files/26179646",  # house_05
    "https://figshare.com/ndownloader/files/26179649",  # house_06
    "https://figshare.com/ndownloader/files/26179658",  # house_07
    "https://figshare.com/ndownloader/files/26179694",  # house_08
    "https://figshare.com/ndownloader/files/26179697",  # house_09
    "https://figshare.com/ndownloader/files/26179700",  # house_10
    "https://figshare.com/ndownloader/files/26179703",  # house_11
    "https://figshare.com/ndownloader/files/26179706",  # house_12
    "https://figshare.com/ndownloader/files/26179709",  # house_13
    "https://figshare.com/ndownloader/files/26179712"   # house_14
]

archives = []

for u in urls:
    archives.append(downloadFile(u))


## Parse appliances data in Pandas dataframes

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Extract data and keep only appliances with annotations

In [5]:
folders = []

for archive in archives:
    with zipfile.ZipFile(archive, "r") as zip:
        zip.extractall()

        folder = archive.split('.')[0]

        items = pd.read_csv(os.path.join(folder, "items.tsv"), sep="\t")
        items_no = 0
        for index, item in items.iterrows():
            labels = pd.read_csv(os.path.join(
                folder, "item_{:04d}_annotation_labels.tsv".format(item.item_id)), sep="\t")
            if len(labels) > 0:
                items_no += 1
            else:
                os.remove(os.path.join(
                    folder, "item_{:04d}_data.tsv.gz".format(item.item_id)))
                os.remove(os.path.join(
                    folder, "item_{:04d}_annotation_labels.tsv".format(item.item_id)))
                os.remove(os.path.join(
                    folder, "item_{:04d}_annotations.tsv".format(item.item_id)))
        if items_no > 0:
            folders.append(folder)
        else:
            for root, dirs, files in os.walk(folder, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(folder)


In [3]:
def makeDir(dirname):
    if not os.path.exists(dirname):
        os.mkdir(dirname)

In [5]:
APPLIANCES_FOLDER = "appliances"


In [7]:
makeDir(APPLIANCES_FOLDER)

for folder in folders:
    items = pd.read_csv(os.path.join(folder, "items.tsv"), sep="\t")
    for index, item in items.iterrows():
        try:
            labels = pd.read_csv(os.path.join(
                folder, "item_{:04d}_annotation_labels.tsv".format(item.item_id)), sep="\t")
            annotations = pd.read_csv(os.path.join(
                folder, "item_{:04d}_annotations.tsv".format(item.item_id)), sep="\t", infer_datetime_format=True)
            annotations.start_date = pd.to_datetime(
                annotations.start_date, format='%Y-%m-%d %H:%M:%S')
            annotations.stop_date = pd.to_datetime(
                annotations.stop_date, format='%Y-%m-%d %H:%M:%S')
            data = pd.read_csv(os.path.join(
                folder, "item_{:04d}_data.tsv.gz".format(item.item_id)), sep="\t", compression="gzip")
            data.time = pd.to_datetime(data.time, format='%Y-%m-%d %H:%M:%S')

            makeDir(os.path.join(APPLIANCES_FOLDER, item.category))
            makeDir(os.path.join(os.path.join(APPLIANCES_FOLDER, item.category),
                                 item.label
                                 ))

            for _, label in labels.iterrows():
                makeDir(os.path.join(
                        os.path.join(
                            os.path.join(APPLIANCES_FOLDER, item.category),
                            item.label),
                        label.text
                        ))

            for _, label in labels.iterrows():
                counter = 0
                for index, annotation in annotations.iterrows():
                    if annotation.label_id == label.id:
                        df = data.loc[(data['time'] >= annotation.start_date)
                                      & (data['time'] < annotation.stop_date)]
                        df.to_csv(os.path.join(
                            os.path.join(
                                os.path.join(
                                    os.path.join(
                                        APPLIANCES_FOLDER, item.category),
                                    item.label),
                                label.text
                            ),
                            str(counter) + ".csv"
                        ))
                        counter += 1
        except Exception as e:
            continue


Remove erroneous data and unlabelled programs

In [23]:
for root, dirs, _ in os.walk(APPLIANCES_FOLDER, topdown=False):
    for dir in dirs:
        for new_root, programs, files in os.walk(os.path.join(APPLIANCES_FOLDER, dir), topdown=False):
            try:
                lens = []
                for file in files:
                    df = pd.read_csv(os.path.join(new_root, file))
                    lens.append(len(df))
                if len(lens) == 0:
                    continue
                len_avg = np.average(lens)
                for file in files:
                    df = pd.read_csv(os.path.join(new_root, file))
                    # maximum variation admitted - 20% from the average program length to remove outliers
                    if np.abs(len_avg - len(df)) > 0.2 * len_avg:
                        os.remove(os.path.join(new_root, file))
                if len(os.listdir(new_root)) == 0:
                    os.rmdir(new_root)
            except:
                continue


In [12]:
for root, dirs, _ in os.walk(APPLIANCES_FOLDER, topdown=False):
    for dir in dirs:
        if dir == "Error" or dir == "Unknown" or dir == "Unkown":
            for root, dirs, files in os.walk(os.path.join(root, dir), topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(os.path.join(root))
        if dir == "unknown activity":
            os.rename(os.path.join(root, dir), os.path.join(root, "Idle"))


Remove archives and unuseful extracted data

In [25]:
for root, dirs, files in os.walk(".", topdown=False):
    for file in files:
        if file[:5] == "house":
            os.remove(os.path.join(root, file))
    for dir in dirs:
        if dir[:5] == "house":
            for root, dirs, files in os.walk(dir, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
                os.rmdir(dir)


Trim unuseful columns

In [18]:
for root, dirs, files in os.walk(APPLIANCES_FOLDER, topdown=False):
    for file in files:
        if file[-4:] == ".csv":
            path = os.path.join(root, file)
            df = pd.read_csv(path)
            if len(df.columns) == 4:
                df["value"].to_csv(path)


# Create consumer background consumption dataset (from non-smart appliances)

## Download files from figshare

In [ ]:
urls = [
    "https://figshare.com/ndownloader/files/28120062",  # 201903
    "https://figshare.com/ndownloader/files/28120176",  # 201904
    "https://figshare.com/ndownloader/files/28120242",  # 201905
    "https://figshare.com/ndownloader/files/28130424",  # 202003
    "https://figshare.com/ndownloader/files/28128384",  # 202004
    "https://figshare.com/ndownloader/files/28130430",  # 202005    
]

archives = []

for u in urls:
    archives.append(downloadFile(u))


In [ ]:
BACKGROUND_FOLDER = "background"
makeDir(BACKGROUND_FOLDER)

ids_complete = []

for archive in archives:
    df = pd.read_csv(archive, compression="gzip")
    df.dropna(inplace=True)
    id_counts = df['id'].value_counts()
    ids_to_keep = id_counts[id_counts > id_counts.max() - 5].index
    df = df.loc[df['id'].isin(ids_to_keep)]
    ids_complete.append(df.id.unique())
    del df


In [ ]:
common_ids = list(set.intersection(*map(set, ids_complete)))
common_ids = np.array(common_ids, dtype=np.int64)
common_ids.sort()
print(common_ids)
print(len(common_ids))
first20 = common_ids[:20]


In [ ]:
for archive in archives:
    year = archive.split("_")[2][:4]
    year_path = os.path.join(BACKGROUND_FOLDER, year)
    makeDir(year_path)
    month = archive.split("_")[2][4:6]
    month_path = os.path.join(year_path, month)
    makeDir(month_path)
    df = pd.read_csv(archive, compression="gzip")
    df.rename(columns={df.columns[0]: 'timestamp'}, inplace=True)
    for index in range(len(first20)):
        df_filtered = df.loc[df['id'] == first20[index]]
        df_filtered.to_csv(os.path.join(month_path, str(index) + ".csv"))
    del df


In [14]:
YEARS = ["2019", "2020"]
MONTHS = ["03", "04", "05"]

In [17]:
to_be_del = set()


for year in YEARS:
    year_path = os.path.join(BACKGROUND_FOLDER, year)
    for month in MONTHS:
        month_path = os.path.join(year_path, month)
        for index in range(20):
            df = pd.read_csv(os.path.join(month_path, str(index) + ".csv"))
            if df['value'].sum() < 50:
                to_be_del.add(index)
            else:
                df['value'] *= 1000
                df[['timestamp', 'value']].to_csv(os.path.join(month_path, str(index) + ".csv"))

for id in to_be_del:
    for year in YEARS:
        year_path = os.path.join(BACKGROUND_FOLDER, year)
        for month in MONTHS:
            month_path = os.path.join(year_path, month)
            os.remove(os.path.join(month_path, str(id) + ".csv"))

Remove archives

In [37]:
for root, dirs, files in os.walk(".", topdown=False):
    for file in files:
        if file[:11] == "consumption":
            os.remove(os.path.join(root, file))


# Create energy production dataset from weather data

In [32]:
WEATHER_FOLDER = "weather"

makeDir(WEATHER_FOLDER)

Read API key from file smart-meter/dataset/.apikey

In [7]:
with open(".apikey", "rt") as f:
    API_KEY = f.readline().strip()


Create temporary email using 1secmail API

In [35]:
import requests
import json
import time
import datetime
import calendar

In [9]:
address_response = requests.get("https://www.1secmail.com/api/v1/?action=genRandomMailbox&count=1")
mail = json.loads(address_response.content)[0]
user = mail.split("@")[0]
domain = mail.split("@")[1]


Create request for data download

In [19]:
for year in YEARS:
    url = "https://developer.nrel.gov/api/nsrdb/v2/solar/msg-iodc-download.csv?api_key=" + API_KEY + \
        "&full_name=Mihaela+Mihaiu&email=mihaela.mihaiu@mta.ro&affiliation=MTA&reason=Academic&mailing_list=false&wkt=POINT(44.57+26.06)&names=" + year + \
        "&attributes=ghi,air_temperature,surface_pressure,relative_humidity,wind_speed&leap_day=false&utc=false&interval=15"

    response = requests.get(url)

    with open(os.path.join(WEATHER_FOLDER, year + "_raw.csv"), "wb") as f:
        f.write(response.content)

    time.sleep(2)   # 2 seconds between API requests


Parse datasets

In [35]:
for year in YEARS:
    df = pd.read_csv(os.path.join(
        WEATHER_FOLDER, year + "_raw.csv"), low_memory=False, skiprows=2)
    df['timestamp'] = pd.to_datetime(
        df[["Year", "Month", "Day", "Hour", "Minute"]])

    Rd = 287.058    # specific gas constant for dry air
    Rv = 461.495    # specific gas constant for water vapor

    df['p1'] = 6.1078 * pow(10, (7.5 * df['Temperature']) /
                            (df['Temperature'] + 273.3))
    df['pv'] = df['p1'] * df['Relative Humidity']
    df['pd'] = df['Pressure'] * 100 - df['pv']
    df['air_density'] = (df['pd'] / (Rd * (df['Temperature'] + 273.3))) + \
        (df['pv'] / (Rv * (df['Temperature'] + 273.3)))   # kg/m3
    df[["timestamp", "GHI", "air_density", "Wind Speed"]].to_csv(os.path.join(
        WEATHER_FOLDER, year + ".csv"))


In [ ]:
for year in YEARS:
    os.remove(os.path.join(
        WEATHER_FOLDER, year + "_raw.csv"))


Specify generators models

In [61]:
# https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems

SOLAR_PANELS_FOLDER = "Solar Panel"

solar_panels = {
    # https://www.europe-solarstore.com/download/lg/LG_Neon_R_Prime_LG355-370Q1K-V5_data_sheet.pdf
    "LG370Q1K-V5": {
        "area": 1.568,
        "efficiency": 21.4,
        "pr": 0.78
    },
    # https://www.europe-solarstore.com/download/sunpower/SunPower-SPR-MAX3-390_Datasheet_EN.pdf
    "SunPower SPR-MAX3-400": {
        "area": 1.659,
        "efficiency": 22.6,
        "pr": 0.75
    },
    # https://www.europe-solarstore.com/download/benq/BenQ_Sunforte_PM096B00_datasheet.pdf
    "BenQ SunForte PM096B00-335": {
        "area": 1.631,
        "efficiency": 19.6,
        "pr": 0.72
    }
}

# Power = A × r × H × PR

WIND_TURBINES_FOLDER = "Wind Turbine"

wind_turbines = {
    "AIR 40": {
        "swept_area": 1.07,
        "efficiency": 0.45
    },
    "Pikasola B08F4SYCF7": {
        "swept_area": 1.14,
        "efficiency": 0.40
    },
    "Raptor G4": {
        "swept_area": 1.70,
        "efficiency": 0.43
    }
}

# Power = 0.5 x Swept Area x Air Density x Velocity3 * efficiency


In [ ]:
solar_path = os.path.join(APPLIANCES_FOLDER, SOLAR_PANELS_FOLDER)

makeDir(solar_path)

for panel in solar_panels:
    panel_path = os.path.join(solar_path, panel)
    makeDir(panel_path)

wind_path = os.path.join(APPLIANCES_FOLDER, WIND_TURBINES_FOLDER)

makeDir(wind_path)

for turbine in wind_turbines:
    turbine_path = os.path.join(wind_path, turbine)
    makeDir(turbine_path)

for year in YEARS:
    df = pd.read_csv(os.path.join(WEATHER_FOLDER, year + ".csv"))
    for month in MONTHS:
        df['timestamp'] = pd.to_datetime(df.timestamp)
        dt = datetime.datetime(year=int(year), month=int(month), day=1)
        start = pd.to_datetime(calendar.timegm(dt.timetuple()), unit='s')
        dt = datetime.datetime(year=int(year), month=int(month)+1, day=1)
        stop = pd.to_datetime(calendar.timegm(dt.timetuple()), unit='s')
        df_month = df.loc[(df['timestamp'] >= start)
                          & (df['timestamp'] < stop)]
        days = np.unique(df_month["timestamp"].dt.date)
        for index in range(len(days) - 1):
            df_day = df_month.loc[(df_month['timestamp'] >=
                                   pd.to_datetime(days[index])) & (df_month['timestamp'] < pd.to_datetime(days[index+1]))]

            for panel in solar_panels:
                panel_path = os.path.join(solar_path, panel)
                panel_path = os.path.join(
                    panel_path, year+month+f"{index+1:02d}"+".csv")
                data = solar_panels[panel]
                values = data["area"] * data["efficiency"] * \
                    data["pr"] * df_day["GHI"]
                df_panel = pd.DataFrame(
                    data={'timestamp': df_day['timestamp'], 'value': values})
                df_panel.to_csv(panel_path)

            for turbine in wind_turbines:
                turbine_path = os.path.join(wind_path, turbine)
                turbine_path = os.path.join(
                    turbine_path, year+month+f"{index+1:02d}"+".csv")
                data = wind_turbines[turbine]
                values = 0.5 * data["swept_area"] * data["efficiency"] * \
                    df_day["air_density"] * pow(df_day["Wind Speed"], 3)
                df_turbine = pd.DataFrame(
                    data={'timestamp': df_day['timestamp'], 'value': values})
                df_turbine.to_csv(turbine_path)
